# Prepare Enviroment

In [3]:
!pip install gspread oauth2client df2gspread
!pip install smartsheet-python-sdk
# Import libraries 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# For connect to google sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from ibm_botocore.client import Config
import ibm_boto3
import smartsheet
import logging
import time
import os
import pandas as pd
from bokeh.plotting import figure, output_file, show,output_notebook
output_notebook()

Loading BokehJS ...

# Update Master Summary from Smartsheets to Google Sheets

### Connect to IBM Cloud Storgae (the google api credentials are store in there in a Json file)

In [4]:
# @hidden_cell
cos_credentials={
  "apikey": "X-MM4GKyMkh95hVzgN0O6s1NGRJlhdmUd3tR4H7wfumM",
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key 8035afca-4460-4b47-9e0f-693a92ec4426",
  "iam_apikey_name": "CredentialsV2",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/648bafda66e84dad8e2ef164829deb7c::serviceid:ServiceId-55437a77-2ab0-4d60-85ad-9df6332e0492",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/648bafda66e84dad8e2ef164829deb7c:e1be1961-729e-4551-a7b5-a04ba4574be6::"
}

auth_endpoint = 'https://iam.bluemix.net/oidc/token'
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'
cos = ibm_boto3.client('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                        ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [5]:
# Google Sheet Credentials 
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials= {
    'IAM_SERVICE_ID': 'iam-ServiceId-a1f7d64c-63d4-4564-822b-a30bf6a8c94b',
    'IBM_API_KEY_ID': 'CgAcXQPKvbVzoTa8IYL_diblsX1juQp3kzL8ju1yooab',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'googlesheetapi-donotdelete-pr-xnwgpdxetbczoi',
    'FILE': 'service_account_GS.json'
}

In [6]:
cos2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [7]:
cos.download_file(Bucket=credentials['BUCKET'], Key = 'service_account_GS.json', Filename='service_account_GS.json')

### Get Smartsheets Master Summary in a Data Frame

In [8]:
from IPython.display import display_html
display_html('https://publish.smartsheet.com/2ff2e15f6ab545d0a505903d68545d79', raw=True)
dfs = pd.read_html('https://publish.smartsheet.com/2ff2e15f6ab545d0a505903d68545d79')
MS_df = dfs[0]
#MS_df.dropna(axis = 0, inplace = True)
MS_df.drop(MS_df[MS_df['GS Last Column'] != '3'].index, inplace = True)
#MS_df[['Keys','Bed Spaces','Commercial Area (SQM)','Total Net Area (SQM)']] = MS_df[['Keys','Bed Spaces','Commercial Area (SQM)','Total Net Area (SQM)']].astype(float)
#MS_df[['IC Presentation Date','IC Approved Date']] = MS_df[['IC Presentation Date','IC Approved Date']].astype('datetime64[ns]')
MS_df.fillna("", inplace = True)
MS_df.drop(['Unnamed: 0','Compress Data', 'Delete'], axis = 1, inplace = True)
#MS_df.set_index(['Project ID (fx)'], inplace = True)
MS_df

https://publish.smartsheet.com/2ff2e15f6ab545d0a505903d68545d79

,Dashboard Link,Project ID (fx),Property ID (fx),Project Commercial Name,Name,Type of deal,Project Status,Region,Country,City,...,Legacy Property ID,Legacy Project ID,GS Last Column,Confirmed Open and Scheduled Open Conflict,Selina Construction Started,Site possesion,Construction Documents sign-off,Business Plan Approval + Sign-off,Concept Documents sign-off,Operating not as a Selina Start Date
0,https://app.smartsheet.com/dashboards/5M8w6prX...,la-bra-sao-137.01,la-bra-sao-137,Selina Madalena São Paulo,Atelier 245 - Vila Madalena,Straight Lease,Active,Latam,Brazil,Sao Paulo,...,La-Bra-Ate-137,La-Bra-Ate-137.01,3,OK,,,,,,
2,https://app.smartsheet.com/dashboards/qv24CcCr...,us-use-nyc-151.01,us-use-nyc-151,Selina Chelsea New York City,Americano,Straight Lease,Active,US,US East,"NYC, NY",...,US-US -Ame-151,US-US -Ame-151.01,3,Different Open Dates,,,,,,
4,https://app.smartsheet.com/dashboards/h77QQ52H...,la-arg-bar-152.01,la-arg-bar-152,Selina Bariloche,Bariloche - Villa Sofia,Straight Lease,Active,Latam,Argentina,Bariloche,...,La-Arg-Bar-152,La-Arg-Bar-152.01,3,OK,,,,,,
6,https://app.smartsheet.com/dashboards/gx9rvRgX...,la-mex-gua-165.01,la-mex-gua-165,Selina Guadalajara,Edificio Chapultepec 2.0 - Moreno,Straight Lease,Active,Latam,Mexico,Guadalajara,...,La-Mex-Edi-165,La-Mex-Edi-165.01,3,OK,,,,,,
8,https://app.smartsheet.com/dashboards/79m63jxV...,us-use-hud-242.01,us-use-hud-242,Selina Woodstock,Woodstock,Acquisition,Active,US,US East,"Hudson Valley, NY",...,US-US -Woo-242,US-US -Woo-242.01,3,Different Open Dates,,,,,,
10,https://app.smartsheet.com/dashboards/CpJWFC6f...,la-arg-cor-163.01,la-arg-cor-163,Selina Nueva Cordoba,Cordoba On Aparts,Straight Lease,Active,Latam,Argentina,Cordoba,...,La-Arg-Cor-163,La-Arg-Cor-163.01,3,OK,,,,,,
12,https://app.smartsheet.com/dashboards/hvfVPPJg...,la-per-are-245.01,la-per-are-245,Selina Arequipa,La Casa de mi Abuelo,Straight Lease,Complete,Latam,Peru,Arequipa,...,La-Per-La -245,La-Per-La -245.01,3,Different Open Dates,,,,,,
14,https://app.smartsheet.com/dashboards/wV53VPvJ...,la-mex-tul-193.01,la-mex-tul-193,Selina Los Lirios Tulum,LOS LIRIOS,Straight Lease,Active,Latam,Mexico,Tulum,...,La-Mex-LOS-193,La-Mex-LOS-193.01,3,Different Open Dates,,,,,,
16,https://app.smartsheet.com/dashboards/MgWGxhFm...,la-mex-san-183.02,la-mex-san-183,Selina San Miguel (RUNNING AS 3rd PARTY),HOTEL VISTA HERMOSA I,Straight Lease,Abandoned,Latam,Mexico,San Miguel de Allende,...,La-Mex-HOT-183,La-Mex-HOT-183.1,3,Different Open Dates,,,,,,
18,https://app.smartsheet.com/dashboards/PGv29GWW...,us-use-nyc-154.01,us-use-nyc-154,Selina Bowery,Bowery,Straight Lease,Abandoned,US,US East,"NYC, NY",...,US-US -Bow-154,US-US -Bow-154.01,3,OK,,,,,,


### Connect to the Spreadsheet

In [11]:
# Configure the connection 
scope = ['https://spreadsheets.google.com/feeds']

# Give the path to the Service Account Credential json file 
credentials = ServiceAccountCredentials.from_json_keyfile_name('service_account_GS.json',
                                                               scope
                                                              )
# Authorise your Notebook
gc = gspread.authorize(credentials)

# The sprad sheet ID, which can be taken from the link to the sheet
spreadsheet_key = '1eF89RXoQeJIjcXfJG6amKnPHPF-WfUJCNkA_2NrVPdM'

In [12]:
from pprint import pprint
from googleapiclient import discovery

In [13]:
# Populate sheet with MS_df DataFrame
wks_name = 'Master Summary (Split)'
cell_of_start_df = 'C1'
d2g.upload(MS_df,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True) #delte content
print ('The sheet is updated successfully')

The sheet is updated successfully
